In [1]:
from datasets import load_dataset

# load iris
iris = load_dataset("scikit-learn/iris")


In [2]:

from jax.numpy import vstack, array


iris_train = iris['train']

iris_train.set_format('jax', columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species'])
X = vstack((iris_train['SepalLengthCm'], iris_train['SepalWidthCm'], iris_train['PetalLengthCm'], iris_train['PetalWidthCm']))
y_str = iris_train['Species']
# y = y_str.map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
# y_str is a list
y = array([0 if s == 'Iris-setosa' else 1 if s == 'Iris-versicolor' else 2 for s in y_str])
y_1hot = array([[1, 0, 0] if s == 'Iris-setosa' else [0, 1, 0] if s == 'Iris-versicolor' else [0, 0, 1] for s in y_str])
y_1hot.shape



(150, 3)

In [5]:
import sys
sys.path.append('../berries')
from positional_encoding import get_positional_encoding
get_positional_encoding(4, 10)


Array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  1.0000000e+00],
       [ 8.4147102e-01,  5.4030222e-01,  1.5782663e-01,  9.8746681e-01,
         2.5116222e-02,  9.9968451e-01,  3.9810599e-03,  9.9999207e-01,
         6.3095725e-04,  9.9999982e-01],
       [ 9.0929747e-01, -4.1614699e-01,  3.1169716e-01,  9.5018154e-01,
         5.0216597e-02,  9.9873835e-01,  7.9620574e-03,  9.9996829e-01,
         1.2619143e-03,  9.9999923e-01],
       [ 1.4112000e-01, -9.8999250e-01,  4.5775455e-01,  8.8907856e-01,
         7.5285286e-02,  9.9716204e-01,  1.1942928e-02,  9.9992865e-01,
         1.8928708e-03,  9.9999821e-01]], dtype=float32, weak_type=True)

In [ ]:
def affine(x, W, b):
    return x @ W + b
from init import zerO_init_2D
W_ = zerO_init_2D((4, 3))
